**BERT model Based on Deep Learning**




BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.Because of large dataset and execution capacity while using jupyter notebook  failed for many experiment and finally I started to work with a small chunk from the oreginal data of df= 1481 rows and 4 columns. Bert model has the following tasks step by step explain as follows:

Task 1: Exploratory Data Analysis and Preprocessing

Task 2: Training/Validation Split

Task 3: Loading Tokenizer and Encoding our Data

Task 4: Setting up BERT Pretrained Model

Task 5: Creating Data Loaders

Task 6: Setting Up Optimizer and Scheduler

Task 7: Defining our Performance Metrics

Task 8: Creating our Training Loop

**Task 1: Exploratory Data Analysis and Preprocessing**

In [ ]:

import torch
import pandas as pd
from tqdm.notebook import tqdm
from  sklearn.model_selection  import  train_test_split

In [ ]:
chunk_size = 10000
batch_num = 1
for chunk in pd.read_csv('/content/train.csv',chunksize=chunk_size):
  chunk.to_csv('train' + str(batch_num) + '.csv', index= False) 
  batch_num ++1
      

In [ ]:
df = pd.read_csv('/content/train1.csv')
df.set_index('textID', inplace=True)

In [ ]:
len(df) # number of observations

7481

In [ ]:
df.selected_text.iloc[0]

'really thinks wee callum should of got through last night'

In [ ]:
df.head(10)

,text,selected_text,sentiment
textID,,,
9e38c183ad,really thinks wee callum should of got through...,really thinks wee callum should of got through...,neutral
d1c010cd47,Once again another late night. But I scored a...,Once again another late night. But I scored a...,neutral
03c0b749b5,Nooo i wanna watch the comet dont wanna go wo...,dont wanna go workk!!!,negative
99f4063cf1,Aw yes we had it out for weeks got asked to ...,! never mind,positive
9d605483af,Mc John sim posit 3 yr passed no soft...,Mc John sim posit 3 yr passed no softees ...,neutral
adede39756,you look smashing darling is trent reznor re...,you look smashing darling is trent reznor rea...,neutral
9655b72897,its bout smoking weed,its bout smoking weed,neutral
52c7993b04,It`s official nobody is getting a raise this year,It`s official nobody is getting a raise this year,negative
2ed155359e,Thank you all for your incredible support and ...,thankful,positive


In [ ]:
df.sentiment.value_counts()

neutral     3051
positive    2268
negative    2162
Name: sentiment, dtype: int64

In [ ]:
print(df.isnull().sum())

text             0
selected_text    0
sentiment        0
dtype: int64


In [ ]:
df.dropna(inplace=True)
print(df.isnull().sum())

text             0
selected_text    0
sentiment        0
dtype: int64


In [ ]:
#df['sentiment_num']= df.sentiment.map({'positive':1,'neutral':0, 'negative':2})

possible_labels = df.sentiment.unique()

In [ ]:
possible_labels

array(['neutral', 'negative', 'positive'], dtype=object)

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
    
    #enumerate loops the introval and returns the index that we are currently on.
    # The first tuple will recieve will be the first possible -lebals. alone side th index 
    #that would be zero and seond is 1 and 3rd 2 and we create a dictonary dict[] and must be set to the index.
    # now add this dtata to the label dict and now add this value to the column label.

In [ ]:
df['label'] = df.sentiment.replace(label_dict)

In [ ]:
df.head(200)

,text,selected_text,sentiment,label
textID,,,,
9e38c183ad,really thinks wee callum should of got through...,really thinks wee callum should of got through...,neutral,0
d1c010cd47,Once again another late night. But I scored a...,Once again another late night. But I scored a...,neutral,0
03c0b749b5,Nooo i wanna watch the comet dont wanna go wo...,dont wanna go workk!!!,negative,1
99f4063cf1,Aw yes we had it out for weeks got asked to ...,! never mind,positive,2
9d605483af,Mc John sim posit 3 yr passed no soft...,Mc John sim posit 3 yr passed no softees ...,neutral,0
...,...,...,...,...
844b2405f4,I`m watching The episode 3 Of JONAS OMJ,I`m watching The episode 3 Of JONAS OMJ,neutral,0
103b305084,Is feelin right rite now,feelin right,positive,2
f7fdea625a,i`m so bored i can barely even tweet. i have n...,bored,negative,1


In [ ]:
df

,text,selected_text,sentiment,label
textID,,,,
9e38c183ad,really thinks wee callum should of got through...,really thinks wee callum should of got through...,neutral,0
d1c010cd47,Once again another late night. But I scored a...,Once again another late night. But I scored a...,neutral,0
03c0b749b5,Nooo i wanna watch the comet dont wanna go wo...,dont wanna go workk!!!,negative,1
99f4063cf1,Aw yes we had it out for weeks got asked to ...,! never mind,positive,2
9d605483af,Mc John sim posit 3 yr passed no soft...,Mc John sim posit 3 yr passed no softees ...,neutral,0
...,...,...,...,...
4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,1
4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,1
f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,2


# Task 3: Training/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25,  #.25
                                                  random_state=17, 
                                                  stratify=df.label.values)


In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['sentiment', 'label', 'data_type']).count()
# nice dataframe that all the data in the list of X_train will retraival.

text  selected_text
sentiment label data_type                     
negative  1     train      1621           1621
                val         541            541
neutral   0     train      2288           2288
                val         763            763
positive  2     train      1701           1701
                val         567            567

In [ ]:
df[df.data_type=='train'].selected_text.values

array(['really thinks wee callum should of got through last night',
       'dont wanna go workk!!!', '! never mind', ..., ', don`t force',
       'Yay good for both of you.', 'But it was worth it  ****.'],
      dtype=object)

# Task 3: Loading Tokenizer and Encoding our Data

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].selected_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].selected_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

5610

In [ ]:
len(dataset_val)

1871

In [ ]:
len(df)

7481

# Task 5: Setting up BERT Pretrained Model

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Task 6: Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 10 #32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Task 7: Setting Up Optimizer and Scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Task 8: Defining our Performance Metrics

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Task 9: Creating our Training Loop

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
     model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5332776628129198
Validation loss: 0.34617412270621417
F1 Score (Weighted): 0.8694751523343955



Epoch 2
Training loss: 0.2894711494943834
Validation loss: 0.3903133689990624
F1 Score (Weighted): 0.8806266194550348



Epoch 3
Training loss: 0.19923969274370162
Validation loss: 0.44232756839668813
F1 Score (Weighted): 0.8801663818224016



Epoch 4
Training loss: 0.14858646519189955
Validation loss: 0.4809418695513159
F1 Score (Weighted): 0.8895254889132972



Epoch 5
Training loss: 0.103441213461597
Validation loss: 0.5122659648954373
F1 Score (Weighted): 0.8856181176739232



In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)
pass

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
#model.load_state_dict(torch.load('<<INSERT MODEL NAME HERE>>.model', map_location=torch.device('cuda')))

In [ ]:
model.load_state_dict(torch.load('/content/finetuned_BERT_epoch_5.model', 	map_location=torch.device('cuda')))


<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: neutral
Accuracy: 665/763

Class: negative
Accuracy: 480/541

Class: positive
Accuracy: 512/567

